In [1]:
import seaborn as sns # for correlation heatmap
import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb
import pushranker
import datetime
import snrf
import importlib
import random
import pickle
import collections
from matplotlib import pyplot as plt
from pathlib import Path
from xgboost import XGBClassifier, XGBRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from typing import List, Dict, Tuple
from multiprocessing import Pool


In [3]:
push_type = 'scheduled' # local, targeted, scheduled
test_day = '0517'

LOCAL_DATA_ROOT = Path('./data/')
# TRAIN_DATA_ROOT = str(LOCAL_DATA_ROOT / 'train' / 'edition=en_US' / f'push_type={push_type}') # / 'dt=2021-04-30-00')
TEST_DATA_ROOT = str(LOCAL_DATA_ROOT / 'test' / 'edition=en_US' / f'push_type={push_type}' / test_day) # / 'dt=2021-05-01-00')

training_format = snrf.package.get_obj_from_name(pushranker, 'survival_feature_spec.binarized_format')

# input_module = pushranker.local_push
input_module = importlib.import_module(f'pushranker.{push_type}_push')

# print(TRAIN_DATA_ROOT)
print(TEST_DATA_ROOT)

data/test/edition=en_US/push_type=scheduled/0517


## Performance Evaluation among DNN rankers

In [ ]:
## DNN push-ranker
dnn_model_name = f'original--{train_day}'
hps = getattr(pushranker.hyperparam, push_type)
dnn_model_1 = pushranker.model.make_model_assembly(hps, input_module.input_spec)
dnn_model_1.hazard_model.load_weights(f'./model/{push_type}-push/DNN/{dnn_model_name}/pushranker')
pushranker.model.initialize_for_training(dnn_model_1, hps)

In [ ]:
dnn_pred = dnn_model_1.training_adapter.predict(test_ds)